In [1]:
with open('merged_data2.csv', 'r') as file:
    merged_data_sample = [next(file) for _ in range(5)]
    
with open('cast_error_data_updated.csv', 'r') as file:
    cast_error_data_sample = [next(file) for _ in range(5)]
    
merged_data_sample, cast_error_data_sample

([',Tid(norsk normaltid),Navn,Stasjon,Tid(norsk normaltid).1,NedbÃ¸r (1 t),Lufttemperatur,Maksimumstemperatur (1 t),Minimumstemperatur (1 t),NedbÃ¸r (12 t),sn_x,time_x,unit_x,Probe temp 429,timestamp_x,sn_y,time_y,unit_y,Temp BALE innL 6480,timestamp_y\n',
  '0,2023-01-06 00:00:00+00:00,Molde - NÃ¸isomhed,SN62290,2023-01-06 00:00:00,0,"7,2","7,3","6,9",,,,,,,,,,,\n',
  '1,2023-01-06 01:00:00+00:00,Molde - NÃ¸isomhed,SN62290,2023-01-06 01:00:00,"0,5","5,4","7,1","5,4",,,,,,,,,,,\n',
  '2,2023-01-06 02:00:00+00:00,Molde - NÃ¸isomhed,SN62290,2023-01-06 02:00:00,"0,1","5,4","5,6","5,3",,,,,,,,,,,\n',
  '3,2023-01-06 03:00:00+00:00,Molde - NÃ¸isomhed,SN62290,2023-01-06 03:00:00,0,"5,4","5,6","5,3",,,,,,,,,,,\n'],
 ['regnr,datostÃ¸pt,datoVrakReg,antall vrak,vrakÃ¥rsak,godkjent,cast error\n',
  '168491,2024-04-18 22:47:45,,0,,True,False\n',
  '168489,2024-04-18 22:47:45,,0,,True,False\n',
  '168488,2024-04-18 22:47:45,,0,,True,False\n',
  '168487,2024-04-18 22:47:45,,0,,True,False\n'])

LOAD DATA

In [2]:
import pandas as pd

# Load the data
merged_data_df = pd.read_csv('merged_data2.csv')
cast_error_data_df = pd.read_csv('cast_error_data_updated.csv')

# Display the first few rows of each dataframe to understand their structure
print("Merged Data:")
print(merged_data_df.head())

print("\nCast Error Data:")
print(cast_error_data_df.head())

Merged Data:
   Unnamed: 0       Tid(norsk normaltid)               Navn  Stasjon  \
0           0  2023-01-06 00:00:00+00:00  Molde - Nøisomhed  SN62290   
1           1  2023-01-06 01:00:00+00:00  Molde - Nøisomhed  SN62290   
2           2  2023-01-06 02:00:00+00:00  Molde - Nøisomhed  SN62290   
3           3  2023-01-06 03:00:00+00:00  Molde - Nøisomhed  SN62290   
4           4  2023-01-06 04:00:00+00:00  Molde - Nøisomhed  SN62290   

  Tid(norsk normaltid).1 Nedbør (1 t) Lufttemperatur  \
0    2023-01-06 00:00:00            0            7,2   
1    2023-01-06 01:00:00          0,5            5,4   
2    2023-01-06 02:00:00          0,1            5,4   
3    2023-01-06 03:00:00            0            5,4   
4    2023-01-06 04:00:00            0            4,4   

  Maksimumstemperatur (1 t) Minimumstemperatur (1 t)  Nedbør (12 t)  sn_x  \
0                       7,3                      6,9            NaN   NaN   
1                       7,1                      5,4           

In [3]:
# Parse the dates correctly
merged_data_df['Tid'] = pd.to_datetime(merged_data_df['Tid(norsk normaltid)'])
cast_error_data_df['datostøpt'] = pd.to_datetime(cast_error_data_df['datostøpt'])

# Select relevant columns
merged_data_df = merged_data_df[['Tid', 'Lufttemperatur', 'Probe temp 429', 'Temp BALE innL 6480']]
cast_error_data_df = cast_error_data_df[['datostøpt', 'cast error']]

# Rename columns for consistency
cast_error_data_df.rename(columns={'datostøpt': 'Tid'}, inplace=True)

In [4]:
# Merge the datasets on the timestamp
combined_df = pd.merge_asof(
    cast_error_data_df.sort_values('Tid'),
    merged_data_df.sort_values('Tid'),
    on='Tid',
    direction='nearest'
)

# Handle any missing values by filling them with the mean
combined_df.fillna(combined_df.mean(), inplace=True)

# Display the merged dataframe
print("Combined Data:")
print(combined_df.head())

MergeError: incompatible merge keys [0] dtype('<M8[ns]') and datetime64[ns, UTC], must be the same type

In [6]:
import pandas as pd

# Load the data
merged_data_df = pd.read_csv('merged_data2.csv')
cast_error_data_df = pd.read_csv('cast_error_data_updated.csv')

# Parse the dates correctly
merged_data_df['Tid'] = pd.to_datetime(merged_data_df['Tid(norsk normaltid)'])
cast_error_data_df['datostøpt'] = pd.to_datetime(cast_error_data_df['datostøpt'])

# Convert timestamps to timezone-naive
merged_data_df['Tid'] = merged_data_df['Tid'].dt.tz_localize(None)
cast_error_data_df['datostøpt'] = cast_error_data_df['datostøpt'].dt.tz_localize(None)

# Select relevant columns
merged_data_df = merged_data_df[['Tid', 'Lufttemperatur', 'Probe temp 429', 'Temp BALE innL 6480']]
cast_error_data_df = cast_error_data_df[['datostøpt', 'cast error']]

# Rename columns for consistency
cast_error_data_df.rename(columns={'datostøpt': 'Tid'}, inplace=True)

# Replace commas with dots and convert to numeric
merged_data_df['Lufttemperatur'] = merged_data_df['Lufttemperatur'].str.replace(',', '.').astype(float)
merged_data_df['Probe temp 429'] = pd.to_numeric(merged_data_df['Probe temp 429'], errors='coerce')
merged_data_df['Temp BALE innL 6480'] = pd.to_numeric(merged_data_df['Temp BALE innL 6480'], errors='coerce')

# Merge the datasets on the timestamp
combined_df = pd.merge_asof(
    cast_error_data_df.sort_values('Tid'),
    merged_data_df.sort_values('Tid'),
    on='Tid',
    direction='nearest'
)

# Handle any missing values by filling them with the mean
combined_df.fillna(combined_df.mean(), inplace=True)

# Split the data into features and target
X = combined_df.drop(columns=['Tid', 'cast error'])
y = combined_df['cast error']

# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the TPOT classifier
from tpot import TPOTClassifier
tpot = TPOTClassifier(verbosity=2, generations=5, population_size=50, random_state=42)
tpot.fit(X_train, y_train)

# Evaluate the model
print("Test Accuracy: ", tpot.score(X_test, y_test))

# Export the best model
tpot.export('best_model.py')


ValueError: could not convert string to float: '-'